# Notebook 3 : Analyse pour la Construction de la Couche Gold

## Objectif

Ce notebook vous guide dans l'**analyse des données de la couche silver** pour préparer la construction de la couche **gold** (analytics/BI/ML).

La couche gold contient des données transformées, nettoyées et optimisées pour l'analyse métier et la création de rapports/dashboards.

## Prérequis

Avant d'exécuter ce notebook, assurez-vous d'avoir :

1. **Exécuté le notebook `2_[LOAD]_load_to_bigquery.ipynb`** pour avoir toutes les tables dans BigQuery (dataset `silver`)
2. **Fichier `.env` configuré** avec les variables d'environnement nécessaires
3. **Service Account** avec les permissions BigQuery (`BigQuery Data Viewer`, `BigQuery Job User`)
4. **Packages Python installés** : `google-cloud-bigquery`, `pandas`, etc.

## Structure du Notebook

Ce notebook contient **4 tâches principales** à réaliser :

1. **Tâche 1 : Analyser la Granularité** - Comprendre le niveau de détail de chaque table
2. **Tâche 2 : Identifier les Transformations** - Déterminer les transformations nécessaires pour la couche gold
3. **Tâche 3 : Identifier les Clés de Jointure** - Mapper les relations entre les tables
4. **Tâche 4 : Analyse métier Identifier les **KPIs** métiers - 

## Résultats Attendus

À la fin de ce notebook, vous devriez avoir :
- Une compréhension claire de la structure et de la granularité de chaque table
- Une liste des transformations à appliquer pour créer la couche gold
- Un schéma de jointures documenté

## Configuration et Connexion à BigQuery

Cette section configure l'environnement et établit la connexion avec BigQuery pour explorer les données de la couche silver.


In [1]:
# Standard library imports
import os
from pathlib import Path

# Third-party imports
import pandas as pd
from dotenv import load_dotenv
from google.cloud import bigquery
from google.oauth2 import service_account

# Configuration
load_dotenv()

ROOT = Path.cwd().parent
PROJECT_ID = os.getenv("PROJECT_ID")
SA_PATH = ROOT / os.getenv("GOOGLE_APPLICATION_CREDENTIALS")
DATASET_ID = "silver"

# Authentification
creds = service_account.Credentials.from_service_account_file(SA_PATH)
bq_client = bigquery.Client(project=PROJECT_ID, credentials=creds)

print(f"[OK] - Connecté au projet: {PROJECT_ID}")
print(f"[OK] - Dataset: {DATASET_ID}")


[OK] - Connecté au projet: big-data-projet-sncf
[OK] - Dataset: silver


---

## Tâche 1 : Analyser la Granularité de Chaque Table

### Objectif

La **granularité** d'une table correspond au niveau de détail des données qu'elle contient. Comprendre la granularité est essentiel pour :
- Déterminer comment agréger les données
- Identifier les duplications potentielles
- Comprendre le niveau de détail nécessaire pour les analyses métier

### Instructions

Pour chaque table du dataset `silver`, vous devez :

1. **Lister les colonnes** et leurs types
2. **Identifier les clés primaires** ou les colonnes qui identifient de manière unique une ligne
3. **Déterminer la granularité** : à quel niveau de détail sont les données ?
   - Exemple : `fact_validations` pourrait être au niveau **jour × gare × type de titre**
4. **Compter les lignes** et estimer la taille des données
5. **Identifier les colonnes de dimension** (références vers d'autres tables)

### Exemple de Format de Réponse

```
Table: dim_gare
- Granularité: 1 ligne = 1 gare
- Clé primaire: id_gares
- Nombre de lignes: 1234
- Colonnes de dimension: aucune (table de dimension)
```


### À Compléter : Analyse de Granularité

**Tables de Dimension :**

1. `dim_gare`
2. `dim_ligne`
3. `dim_arret`
4. `dim_vacances_scolaires`
5. `dim_transporteur`

**Tables de Fait :**

6. `fact_validations_*` (toutes les tables de validation)

**Votre tâche :** Exécutez des requêtes SQL pour analyser chaque table et remplir le tableau ci-dessous.


In [ ]:
# Exemple : Analyser la table dim_gare
query = f"""
SELECT
  COUNT(*) AS nb_lignes,
  COUNT(DISTINCT id_gares) AS nb_gares_uniques,
  COUNTIF(id_gares IS NULL) AS nb_id_null
FROM `{PROJECT_ID}.{DATASET_ID}.gares`

"""

df = bq_client.query(query).to_dataframe()
print("=== Analyse de dim_gare ===")
display(df)

# Afficher le schéma
table = bq_client.get_table(f"{PROJECT_ID}.{DATASET_ID}.gares")
print("\n=== Schéma ===")
for field in table.schema:
    print(f"  - {field.name}: {field.field_type} ({field.mode})")


=== Analyse de dim_gare ===


c:\Users\moham\Master 2 - CS\BigData\ProjetFinal\m2-univ-reims-sep-cs-etl-sncf-gcp\venv\Lib\site-packages\google\cloud\bigquery\table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,nb_lignes,nb_gares_uniques,nb_id_null
0,1240,1237,0



=== Schéma ===
  - geo_point_2d: GEOGRAPHY (NULLABLE)
  - geo_shape: GEOGRAPHY (NULLABLE)
  - id_gares: INTEGER (REQUIRED)
  - nom_gares: STRING (NULLABLE)
  - nom_so_gar: STRING (NULLABLE)
  - nom_su_gar: STRING (NULLABLE)
  - id_ref_zdc: INTEGER (NULLABLE)
  - nom_zdc: STRING (NULLABLE)
  - id_ref_zda: INTEGER (NULLABLE)
  - nom_zda: STRING (NULLABLE)
  - idrefliga: STRING (NULLABLE)
  - idrefligc: STRING (NULLABLE)
  - res_com: STRING (NULLABLE)
  - indice_lig: STRING (NULLABLE)
  - mode: STRING (NULLABLE)
  - tertrain: STRING (NULLABLE)
  - terrer: STRING (NULLABLE)
  - termetro: STRING (NULLABLE)
  - tertram: STRING (NULLABLE)
  - terval: STRING (NULLABLE)
  - exploitant: STRING (NULLABLE)
  - idf: INTEGER (NULLABLE)
  - principal: INTEGER (NULLABLE)
  - x: FLOAT (NULLABLE)
  - y: FLOAT (NULLABLE)
  - picto: STRING (NULLABLE)
  - nom_iv: STRING (NULLABLE)


### Analyse – table gares

La table gares contient les informations descriptives sur les gares de SNCF.
Chaque gare est identifiée par la colonne id_gares.

On observe :
- 1240 lignes au total
- 1237 identifiants de gares distincts
- aucune valeur NULL sur la colonne id_gares

La différence entre le nombre de lignes et le nombre de gares distinctes indique la présence de quelques doublons.
La granularité de cette table est donc : **1 ligne = 1 gare**.

Il s’agit d’une table de dimension.


In [21]:
table = bq_client.get_table(f"{PROJECT_ID}.{DATASET_ID}.dim_ligne")

for field in table.schema:
    print(field.name, field.field_type)

id_line STRING
name_line STRING
shortname_line STRING
transportmode STRING
transportsubmode STRING
type STRING
operatorref STRING
operatorname STRING
additionaloperators STRING
networkname STRING
colourweb_hexa STRING
textcolourweb_hexa STRING
colourprint_cmjn STRING
textcolourprint_hexa STRING
accessibility STRING
audiblesigns_available STRING
visualsigns_available STRING
id_groupoflines STRING
shortname_groupoflines STRING
notice_title STRING
notice_text STRING
picto STRING
valid_fromdate DATE
valid_todate DATE
status STRING
privatecode STRING
air_conditioning STRING
id_bus_contrat INTEGER


In [8]:
# Analyser la table dim_ligne
query = f"""
SELECT
  COUNT(*) AS nb_lignes,
  COUNT(DISTINCT id_line) AS nb_lignes_uniques,
  COUNTIF(id_line IS NULL) AS nb_id_null
FROM `{PROJECT_ID}.{DATASET_ID}.dim_ligne`

"""

df = bq_client.query(query).to_dataframe()
print("=== Analyse de dim_ligne ===")
display(df)

# Afficher le schéma
table = bq_client.get_table(f"{PROJECT_ID}.{DATASET_ID}.dim_ligne")
print("\n=== Schéma ===")
for field in table.schema:
    print(f"  - {field.name}: {field.field_type} ({field.mode})")

c:\Users\moham\Master 2 - CS\BigData\ProjetFinal\m2-univ-reims-sep-cs-etl-sncf-gcp\venv\Lib\site-packages\google\cloud\bigquery\table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


=== Analyse de dim_ligne ===


,nb_lignes,nb_lignes_uniques,nb_id_null
0,2116,2116,0



=== Schéma ===
  - id_line: STRING (NULLABLE)
  - name_line: STRING (NULLABLE)
  - shortname_line: STRING (NULLABLE)
  - transportmode: STRING (NULLABLE)
  - transportsubmode: STRING (NULLABLE)
  - type: STRING (NULLABLE)
  - operatorref: STRING (NULLABLE)
  - operatorname: STRING (NULLABLE)
  - additionaloperators: STRING (NULLABLE)
  - networkname: STRING (NULLABLE)
  - colourweb_hexa: STRING (NULLABLE)
  - textcolourweb_hexa: STRING (NULLABLE)
  - colourprint_cmjn: STRING (NULLABLE)
  - textcolourprint_hexa: STRING (NULLABLE)
  - accessibility: STRING (NULLABLE)
  - audiblesigns_available: STRING (NULLABLE)
  - visualsigns_available: STRING (NULLABLE)
  - id_groupoflines: STRING (NULLABLE)
  - shortname_groupoflines: STRING (NULLABLE)
  - notice_title: STRING (NULLABLE)
  - notice_text: STRING (NULLABLE)
  - picto: STRING (NULLABLE)
  - valid_fromdate: DATE (NULLABLE)
  - valid_todate: DATE (NULLABLE)
  - status: STRING (NULLABLE)
  - privatecode: STRING (NULLABLE)
  - air_conditio

### Analyse – table dim_ligne

La table dim_ligne contient les informations descriptives des lignes de transport
(métro, RER, bus, tram, etc.).

Chaque ligne est identifiée par la colonne id_ligne.

On observe :
- 2116 lignes au total
- 2116 lignes distinctes
- aucune valeur NULL sur l’identifiant

La granularité de cette table est donc :
**1 ligne = 1 ligne de transport**.

Il s’agit d’une table de dimension.

In [20]:
table = bq_client.get_table(f"{PROJECT_ID}.{DATASET_ID}.dim_arret")

for field in table.schema:
    print(field.name, field.field_type)

arrid STRING
arrversion STRING
arrcreated TIMESTAMP
arrchanged TIMESTAMP
arrname STRING
arrtype STRING
arrxepsg2154 INTEGER
arryepsg2154 INTEGER
arrtown STRING
arrpostalregion STRING
arraccessibility STRING
arraudiblesignals STRING
arrvisualsigns STRING
arrfarezone STRING
zdaid STRING
arrgeopoint GEOGRAPHY


In [11]:
# Analyser la table dim_arrêt
query = f"""
SELECT
  COUNT(*) AS nb_lignes,
  COUNT(DISTINCT arrid) AS nb_arret_uniques,
  COUNTIF(arrid IS NULL) AS nb_id_null
FROM `{PROJECT_ID}.{DATASET_ID}.dim_arret`

"""

df = bq_client.query(query).to_dataframe()
print("=== Analyse de dim_arrêt ===")
display(df)

# Afficher le schéma
table = bq_client.get_table(f"{PROJECT_ID}.{DATASET_ID}.dim_arret")
print("\n=== Schéma ===")
for field in table.schema:
    print(f"  - {field.name}: {field.field_type} ({field.mode})")

c:\Users\moham\Master 2 - CS\BigData\ProjetFinal\m2-univ-reims-sep-cs-etl-sncf-gcp\venv\Lib\site-packages\google\cloud\bigquery\table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


=== Analyse de dim_arrêt ===


,nb_lignes,nb_arret_uniques,nb_id_null
0,38368,38368,0



=== Schéma ===
  - arrid: STRING (NULLABLE)
  - arrversion: STRING (NULLABLE)
  - arrcreated: TIMESTAMP (NULLABLE)
  - arrchanged: TIMESTAMP (NULLABLE)
  - arrname: STRING (NULLABLE)
  - arrtype: STRING (NULLABLE)
  - arrxepsg2154: INTEGER (NULLABLE)
  - arryepsg2154: INTEGER (NULLABLE)
  - arrtown: STRING (NULLABLE)
  - arrpostalregion: STRING (NULLABLE)
  - arraccessibility: STRING (NULLABLE)
  - arraudiblesignals: STRING (NULLABLE)
  - arrvisualsigns: STRING (NULLABLE)
  - arrfarezone: STRING (NULLABLE)
  - zdaid: STRING (NULLABLE)
  - arrgeopoint: GEOGRAPHY (NULLABLE)


### Analyse – table dim_arret

La table **dim_arret** contient les informations descriptives sur les arrêts du réseau.

Chaque arrêt est identifié par la colonne arrid.

On observe :
- 38 368 lignes au total
- 38 368 arrêts distincts
- aucune valeur NULL sur l’identifiant

Le nombre de lignes est égal au nombre d’identifiants distincts, ce qui veut dire qu'il n'y a aucun doublons ici.

La granularité de cette table est :
**1 ligne = 1 arrêt**.

Il s’agit d’une table de dimension.

In [18]:
table = bq_client.get_table(f"{PROJECT_ID}.{DATASET_ID}.dim_vacances_scolaires")

for field in table.schema:
    print(field.name, field.field_type)

description STRING
population STRING
start_date TIMESTAMP
end_date TIMESTAMP
location STRING
zones STRING
annee_scolaire STRING


In [14]:
# Analyser la table dim_vacances_scolaires
query = f"""
SELECT
  COUNT(*) AS nb_lignes
FROM `{PROJECT_ID}.{DATASET_ID}.dim_vacances_scolaires`
"""

df = bq_client.query(query).to_dataframe()
print("=== Analyse de dim_vacances_scolaires ===")
display(df)

# Afficher le schéma
table = bq_client.get_table(f"{PROJECT_ID}.{DATASET_ID}.dim_vacances_scolaires")
print("\n=== Schéma ===")
for field in table.schema:
    print(f"  - {field.name}: {field.field_type} ({field.mode})")

=== Analyse de dim_vacances_scolaires ===


c:\Users\moham\Master 2 - CS\BigData\ProjetFinal\m2-univ-reims-sep-cs-etl-sncf-gcp\venv\Lib\site-packages\google\cloud\bigquery\table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,nb_lignes
0,2320



=== Schéma ===
  - description: STRING (NULLABLE)
  - population: STRING (NULLABLE)
  - start_date: TIMESTAMP (NULLABLE)
  - end_date: TIMESTAMP (NULLABLE)
  - location: STRING (NULLABLE)
  - zones: STRING (NULLABLE)
  - annee_scolaire: STRING (NULLABLE)


### Analyse – table dim_vacances_scolaires

La table dim_vacances_scolaires contient des informations sur les périodes de vacances scolaires.

Chaque ligne correspond à une période de vacances pour une zone donnée et une année scolaire.

On observe :
- 2 320 lignes au total

Les principales informations présentes dans la table sont :
- le type de vacances (description)
- la population concernée
- la date de début et la date de fin
- la zone géographique
- l’année scolaire

Il n’existe pas d’identifiant unique simple pour cette table.
L’identification d’une ligne se fait en combinant plusieurs colonnes
(zone, dates et année scolaire).

La granularité est donc :
**1 ligne = 1 période de vacances × 1 zone × 1 année scolaire**.

Il s’agit d’une table de dimension temporelle.

In [16]:
table = bq_client.get_table(f"{PROJECT_ID}.{DATASET_ID}.dim_transporteur")

for field in table.schema:
    print(field.name, field.field_type)

operatorname STRING
operatorref STRING
housenumber STRING
street STRING
addressline1 STRING
town STRING
postcode STRING
postcodeextension STRING
phone STRING
url STRING
furtherdetails STRING
contactperson STRING
logo STRING
email STRING


In [17]:
# Analyser la table dim_transporteur
query = f"""
SELECT
  COUNT(*) AS nb_lignes,
  COUNT(DISTINCT operatorref) AS nb_transporteurs_uniques,
  COUNTIF(operatorref IS NULL) AS nb_id_null
FROM `{PROJECT_ID}.{DATASET_ID}.dim_transporteur`

"""

df = bq_client.query(query).to_dataframe()
print("=== Analyse de dim_transporteur ===")
display(df)

# Afficher le schéma
table = bq_client.get_table(f"{PROJECT_ID}.{DATASET_ID}.dim_transporteur")
print("\n=== Schéma ===")
for field in table.schema:
    print(f"  - {field.name}: {field.field_type} ({field.mode})")

=== Analyse de dim_transporteur ===


c:\Users\moham\Master 2 - CS\BigData\ProjetFinal\m2-univ-reims-sep-cs-etl-sncf-gcp\venv\Lib\site-packages\google\cloud\bigquery\table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,nb_lignes,nb_transporteurs_uniques,nb_id_null
0,54,54,0



=== Schéma ===
  - operatorname: STRING (NULLABLE)
  - operatorref: STRING (NULLABLE)
  - housenumber: STRING (NULLABLE)
  - street: STRING (NULLABLE)
  - addressline1: STRING (NULLABLE)
  - town: STRING (NULLABLE)
  - postcode: STRING (NULLABLE)
  - postcodeextension: STRING (NULLABLE)
  - phone: STRING (NULLABLE)
  - url: STRING (NULLABLE)
  - furtherdetails: STRING (NULLABLE)
  - contactperson: STRING (NULLABLE)
  - logo: STRING (NULLABLE)
  - email: STRING (NULLABLE)


### Analyse – table dim_transporteur

La table dim_transporteur contient les informations descriptives sur les transporteurs.

Chaque transporteur est identifié de manière logique par la colonne `operatorref`,
qui correspond à un code opérateur.

On observe :
- 54 lignes au total
- 54 transporteurs distincts
- aucune valeur NULL sur l’identifiant

Le nombre de lignes est égal au nombre de transporteurs distincts, pas de doublons.

Il n’existe pas de clé primaire technique explicite dans cette table.
La colonne `operatorref` est utilisée comme identifiant fonctionnel.

La granularité de cette table est :
**1 ligne = 1 transporteur**.

Il s’agit d’une table de dimension.

---

## Tâche 2 : Identifier les Transformations Nécessaires

### Objectif

Identifier les **transformations** à appliquer aux données de la couche silver pour créer la couche gold optimisée pour l'analyse.

### Types de Transformations Possibles

1. **Nettoyage des données**
   - Suppression des doublons
   - Gestion des valeurs NULL
   - Normalisation des formats (dates, textes)

2. **Enrichissement**
   - Ajout de colonnes calculées
   - Jointures avec les tables de dimension
   - Ajout de catégories/segments

3. **Agrégation**
   - Regroupement par dimensions (jour, gare, ligne, etc.)
   - Calcul de métriques (somme, moyenne, comptage)
   - Création de tables pré-agrégées

### Instructions

Pour chaque table, identifiez :
1. **Les problèmes de qualité** à corriger
2. **Les transformations nécessaires** avec des exemples concrets
3. **Les colonnes à ajouter** (calculées ou issues de jointures)
4. **Les agrégations possibles** pour optimiser les requêtes


In [23]:
table = bq_client.get_table(f"{PROJECT_ID}.{DATASET_ID}.fact_validations_2023_s2_nb_fer_txt")

for field in table.schema:
    print(field.name, field.field_type)

JOUR DATE
CODE_STIF_TRNS INTEGER
CODE_STIF_RES INTEGER
CODE_STIF_ARRET INTEGER
LIBELLE_ARRET STRING
ID_ZDC INTEGER
CATEGORIE_TITRE STRING
NB_VALD INTEGER


**Transformation à prévoir pour la couche gold :**

À partir du schéma de la table, plusieurs transformations sont nécessaires pour construire la couche gold.

**1. Nettoyage des données**

- Vérifier les éventuels doublons pour une même combinaison (jour, gare (ID_ZDC), catégorie de titre)

- Vérifier la présence de valeurs NULL dans les colonnes importantes (JOUR, ID_ZDC, NB_VALD)

- Supprimer ou ignorer les lignes où le nombre de validations est manquant

Cela permet d’éviter de compter plusieurs fois les mêmes validations.

**2. Normalisation et préparation des dates**

- Extraire à partir de la colonne **JOUR** :
  - l’année  
  - le mois  
  - le jour de la semaine  

Ces informations permettront d’analyser l’évolution des validations dans le temps, par exemple par mois ou par jour.

**3. Enrichissement par jointures**

- Joindre la table avec les tables de dimension :
    - dim_gare via ID_ZDC
    - dim_ligne via CODE_SITE_LIGNE

Cela permet d’obtenir des informations lisibles (nom de la gare, nom de la ligne) au lieu d’identifiants techniques.

L'objectif final est d'obtenir une `table gold` plus lisible, plus propre et directement exploitable pour les tableaux de bord et les analyses métier.

In [22]:
# Exemple 1 : Vérifier les doublons dans fact_validations

#fact_validations_2023
TABLE_FACT = "fact_validations_2023_s2_nb_fer_txt"

query = f"""
SELECT
  JOUR,
  ID_ZDC,
  CATEGORIE_TITRE,
  COUNT(*) AS nb_occurrences
FROM `{PROJECT_ID}.{DATASET_ID}.{TABLE_FACT}`
GROUP BY 1,2,3
HAVING COUNT(*) > 1
ORDER BY nb_occurrences DESC
LIMIT 20
"""

df_duplicates = bq_client.query(query).to_dataframe()
print("=== Exemple : Recherche de doublons ===")
display(df_duplicates)


c:\Users\moham\Master 2 - CS\BigData\ProjetFinal\m2-univ-reims-sep-cs-etl-sncf-gcp\venv\Lib\site-packages\google\cloud\bigquery\table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


=== Exemple : Recherche de doublons ===


,JOUR,ID_ZDC,CATEGORIE_TITRE,nb_occurrences
0,2023-07-14,73626,Amethyste,3
1,2023-07-04,73626,Amethyste,3
2,2023-07-10,73626,Amethyste,3
3,2023-07-09,73626,Amethyste,3
4,2023-07-13,73626,Amethyste,3
5,2023-07-17,73626,Amethyste,3
6,2023-07-16,73626,Amethyste,3
7,2023-07-02,73626,Amethyste,3
8,2023-07-06,73626,Amethyste,3
9,2023-07-05,73626,Amethyste,3


On observe des doublons dans la table de validations : certaines combinaisons
(JOUR, ID_ZDC, CATEGORIE_TITRE) apparaissent plusieurs fois.

Exemple : pour ID_ZDC = 73626 et CATEGORIE_TITRE = "Amethyste",
plusieurs jours ont nb_occurrences = 3, ce qui signifie que la même information
est présente 3 fois dans la table.

In [25]:
# Exemple 2 : Vérifier les valeurs NULL

#fact_validations_2023
TABLE_FACT = "fact_validations_2023_s2_nb_fer_txt"

query = f"""
SELECT
  COUNTIF(JOUR IS NULL) AS nb_jour_null,
  COUNTIF(ID_ZDC IS NULL) AS nb_id_zdc_null,
  COUNTIF(CATEGORIE_TITRE IS NULL) AS nb_categorie_null,
  COUNTIF(NB_VALD IS NULL) AS nb_nb_valid_null
FROM `{PROJECT_ID}.{DATASET_ID}.{TABLE_FACT}`
"""

df_nulls = bq_client.query(query).to_dataframe()
print("=== Exemple : Vérification des valeurs NULL ===")
display(df_nulls)


c:\Users\moham\Master 2 - CS\BigData\ProjetFinal\m2-univ-reims-sep-cs-etl-sncf-gcp\venv\Lib\site-packages\google\cloud\bigquery\table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


=== Exemple : Vérification des valeurs NULL ===


,nb_jour_null,nb_id_zdc_null,nb_categorie_null,nb_nb_valid_null
0,0,0,0,0


Nous avons réalisés une vérification des valeurs manquantes sur les colonnes principales de la table de faits.

Colonnes analysées :
- JOUR
- ID_ZDC
- CATEGORIE_TITRE
- NB_VALD

Résultats :
- Aucune valeur NULL n’a été détectée sur ces colonnes.
- Toutes les lignes possèdent des informations complètes.

Conclusion :
La table ne présente pas de problème de valeurs manquantes sur les champs clés.
Aucune action de nettoyage n’est nécessaire à ce niveau pour la couche gold.


In [27]:
# Exemple 3 : Agrégation des validations

TABLE_FACT = "fact_validations_2023_s2_nb_fer_txt"

query = f"""
SELECT
  JOUR,
  CATEGORIE_TITRE,
  SUM(NB_VALD) AS total_validations
FROM `{PROJECT_ID}.{DATASET_ID}.{TABLE_FACT}`
GROUP BY JOUR, CATEGORIE_TITRE
ORDER BY JOUR
LIMIT 20
"""

df_aggregation = bq_client.query(query).to_dataframe()
print("=== Exemple : Agrégation des validations ===")
display(df_aggregation)

c:\Users\moham\Master 2 - CS\BigData\ProjetFinal\m2-univ-reims-sep-cs-etl-sncf-gcp\venv\Lib\site-packages\google\cloud\bigquery\table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


=== Exemple : Agrégation des validations ===


,JOUR,CATEGORIE_TITRE,total_validations
0,2023-07-01,Amethyste,91125
1,2023-07-01,Autres titres,313944
2,2023-07-01,NON DEFINI,94811
3,2023-07-01,Forfaits courts,19618
4,2023-07-01,Forfait Navigo,1545278
5,2023-07-01,Imagine R,704032
6,2023-07-01,Contrat Solidarité Transport,458234
7,2023-07-02,Contrat Solidarité Transport,360732
8,2023-07-02,NON DEFINI,76797
9,2023-07-02,Forfait Navigo,1140230


Enfin, nous avons réalisés une agrégation sur la table des validations pour analyser le volume total de validations
par **jour** et par **catégorie de titre**.

Les données ont été regroupées selon :
- la date (JOUR)
- la catégorie de titre (CATEGORIE_TITRE)

Pour chaque groupe, nous avons calculés le nombre total de validations.

Résultats observés :
- Certaines catégories comme `Forfait Navigo` et `Imagine R` présentent un volume de validations très élevé.
- Les volumes varient fortement d’un jour à l’autre.
- Cette agrégation permet d’identifier les titres les plus utilisés selon la période.

Pour conclure, cette transformation est pertinente pour la couche gold, car elle permet de produire des indicateurs
simples et exploitables pour l’analyse métier (suivi journalier, comparaison entre titres, tableaux de bord).


---

## Tâche 3 : Identifier les Clés de Jointure Possibles

### Objectif

Identifier toutes les **relations possibles** entre les tables pour pouvoir créer des jointures dans la couche gold.

### Instructions

Pour chaque paire de tables, identifiez :

1. **Les colonnes de jointure** (clés étrangères)
2. **Le type de relation** (1-1, 1-N, N-N)
3. **La cardinalité** (combien de lignes de la table A correspondent à combien de lignes de la table B)
4. **Vérifier l'intégrité référentielle** (toutes les clés étrangères existent-elles dans la table de dimension ?)


**1. Jointure fact_validations VS dim_gare**

- Clé de jointure :
  - fact_validations.ID_ZDC
  - dim_gare.id_ref_zdc

- Type de relation : 1–N  
  une gare peut avoir plusieurs validations

- Cardinalité :
  - 1 gare → N validations

- Intégrité référentielle :
  Les identifiants `ID_ZDC` présents dans la table de faits doivent exister dans la table `dim_gare`.


**2. Jointure fact_validations VS dim_ligne**

- Clé de jointure :
  - fact_validations.CODE_STIF_LIGNE
  - dim_ligne.id_ligne

- Type de relation : 1–N

- Cardinalité :
  - 1 ligne → plusieurs validations

- Intégrité référentielle :
  Les codes de ligne présents dans la table de faits doivent exister dans la table `dim_ligne`.


**3. Jointure fact_validations VS dim_arret**

- Clé de jointure :
  - fact_validations.CODE_STIF_ARRET
  - dim_arret.arrid

- Type de relation : 1–N

- Cardinalité :
  - 1 arrêt → plusieurs validations

- Intégrité référentielle :
  Les identifiants d’arrêt présents dans la table de faits doivent exister dans `dim_arret`.


**3. Jointure fact_validations VS dim_vacances_scolaires**

- Clé de jointure :
  - fact_validations.JOUR
  - dim_vacances_scolaires.start_date / end_date

- Type de relation : N–1

- Cardinalité :
  - plusieurs validations peuvent appartenir à une même période scolaire

- Intégrité référentielle :
  La date JOUR doit être comprise dans une période définie dans la table `dim_vacances_scolaires`.


**4. Jointure fact_validations VS dim_transporteur**

- Clé de jointure :
  - fact_validations.operatorref
  - dim_transporteur.operatorref

- Type de relation : 1–N

- Cardinalité :
  - un transporteur gère plusieurs validations

- Intégrité référentielle :
  Les opérateurs présents dans la table de faits doivent exister dans `dim_transporteur`.


---

## Tâche 4 : Analyse métier

Identifier les **KPIs** (Key Performance Indicators) à calculer :
- Nombre total de validations par période
- Répartition par type de titre
- Top 10 des gares les plus fréquentées
- Comparaison jour ouvrable vs weekend



Ici, nous identifierons les indicateurs clés (KPI) qui peuvent être utilisés pour analyser l’utilisation du réseau SNCF.

**1. Nombre total de validations par période**

On peut calculer le nombre total de validations par:
- jour
- mois
- année

Cela permet d’observer l’évolution de la fréquentation dans le temps 
et d’identifier les périodes de forte ou faible affluence.


**2. Répartition des validations par type de titre**

Il est possible d’analyser les validations selon le type de titre 
(Forfait Navigo, etc.).

Cette analyse permet de comprendre par exemple:
- quels titres sont les plus utilisés
- quels types d’usagers utilisent le réseau


**3. Les 10 gares les plus fréquentées**

On peut identifier les 10 gares qui ont le plus grand nombre de validations.

Cela permet par exemple:
- de repérer les gares les plus fréquentées
- d’aider à la gestion du trafic et des infrastructures

**4. Comparaison jours ouvrables vs week-end**

Il est possible de comparer le nombre de validations :
- les jours de semaine
- les week-ends

Cette comparaison permet de mieux comprendre les comportements des usagers 
selon quel jour on est.